## Домашнее задание

все материалы для выполения дз в `sem2.ipynb`


### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('./sample_data/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('./sample_data/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [5]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

from nltk import ngrams
import nltk
from nltk import collocations 
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from collections import Counter

In [6]:
nltk.download('genesis')
nltk.download('stopwords')
nltk.download('punkt')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)

In [8]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2870536


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [9]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69472),
 ('и', 55166),
 ('в', 52902),
 ('я', 52818),
 ('RT', 38070),
 ('на', 35759),
 ('http', 32998),
 ('что', 31541),
 ('с', 27217),
 ('а', 26860)]

####Задание 1

In [10]:
# Создадим три группы токенов: с высокой, средней и низкой частотой

high_tokens = set()
medium_tokens = set()
low_tokens = set()

for i in freq_dict_sorted:
    if i[1] > 8000:
        high_tokens.add(i[0])
    elif i[1] < 500:
        low_tokens.add(i[0])
    else:
        medium_tokens.add(i[0])

print(len(high_tokens),len(medium_tokens),len(low_tokens))  

34 502 350587


In [11]:
def MyFit_Report (stop_words_list):

    vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stop_words_list)
    bow = vec.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))

    print(classification_report(pred, y_test))

In [12]:
# токены с высокой частотой
%%time
MyFit_Report(noise + list(low_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.80      0.58      0.67     39011
    positive       0.43      0.69      0.53     17698

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.69      0.61      0.63     56709

CPU times: user 1min 22s, sys: 5.96 s, total: 1min 28s
Wall time: 1min 23s


In [13]:
# токены со средней частотой
%%time
MyFit_Report(noise + list(high_tokens) + list(low_tokens))

              precision    recall  f1-score   support

    negative       0.56      0.69      0.62     23070
    positive       0.75      0.64      0.69     33639

    accuracy                           0.66     56709
   macro avg       0.66      0.66      0.65     56709
weighted avg       0.67      0.66      0.66     56709

CPU times: user 1min 24s, sys: 6.77 s, total: 1min 31s
Wall time: 1min 26s


In [14]:
# токены с низкой частотой
%%time
MyFit_Report(noise + list(high_tokens) + list(medium_tokens))

              precision    recall  f1-score   support

    negative       0.79      0.72      0.76     30778
    positive       0.70      0.78      0.74     25931

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

CPU times: user 1min 1s, sys: 10.9 s, total: 1min 12s
Wall time: 1min 2s


По данной метрике лучшая точность получается по низкочастотным токенам. Если смещать границы - точность по другим токенам будет увеличиваться. Данная метрика зависит от количества токенов в большей степени.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

####Задание 2
Как мы увидели из ноутбука на вебинаре -- среди пунктуации могут оказаться значимые фичи. Попробуем с ними поработать и вывести наиболее значимые.

In [16]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27995
    positive       1.00      1.00      1.00     28714

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [17]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
cool_token = '@'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.57      0.56      0.57     28363
    positive       0.57      0.57      0.57     28346

    accuracy                           0.57     56709
   macro avg       0.57      0.57      0.57     56709
weighted avg       0.57      0.57      0.57     56709



In [19]:
cool_token = '&'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.99      0.49      0.66     56366
    positive       0.01      0.53      0.01       343

    accuracy                           0.50     56709
   macro avg       0.50      0.51      0.34     56709
weighted avg       0.99      0.50      0.66     56709



In [20]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32886
    positive       0.83      1.00      0.91     23823

    accuracy                           0.91     56709
   macro avg       0.92      0.93      0.91     56709
weighted avg       0.93      0.91      0.92     56709



In [21]:
tokens_arr = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    tokens_arr.append((i,accuracy_score(pred, y_test)))
print(tokens_arr)

[('!', 0.5163906963621294), ('"', 0.5081733058244723), ('#', 0.5047875998518754), ('$', 0.49484208855737183), ('%', 0.4975929746601069), ('&', 0.4950360612953852), ("'", 0.4946833835899064), ('(', 0.026468461796187556), (')', 0.9147930663563103), ('*', 0.5141159251617909), ('+', 0.495723782821069), (',', 0.5048228676224232), ('-', 0.5090197323176215), ('.', 0.5063041139854344), ('/', 0.5456100442610521), (':', 0.5455571426052302), (';', 0.5007670740094164), ('<', 0.4947010174751803), ('=', 0.49551217619778165), ('>', 0.4947010174751803), ('?', 0.5054047858364633), ('@', 0.5682343190675202), ('[', 0.4945775802782627), ('\\', 0.49471865136045423), (']', 0.4946657497046324), ('^', 0.5008905112063341), ('_', 0.5181717187747976), ('`', 0.4944188753107972), ('{', 0.4946657497046324), ('|', 0.49046888500943414), ('}', 0.4947010174751803), ('~', 0.4947010174751803)]


####Задание 3

In [23]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.64      0.78     43987
    positive       0.44      1.00      0.61     12722

    accuracy                           0.72     56709
   macro avg       0.72      0.82      0.70     56709
weighted avg       0.88      0.72      0.74     56709

CPU times: user 1min 4s, sys: 11.6 s, total: 1min 16s
Wall time: 1min 7s


In [24]:
%%time

tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27995
    positive       1.00      1.00      1.00     28714

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709

CPU times: user 1min 5s, sys: 12.4 s, total: 1min 17s
Wall time: 1min 6s


In [25]:
from sklearn.feature_extraction.text import HashingVectorizer

In [29]:
for n in range (4, 20, 2):
    vectorizer = HashingVectorizer(n_features=2**n,)
    bow = vectorizer.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    pred = clf.predict(vectorizer.transform(x_test))
    print(f'n_features = {2**n}')
    print(classification_report(pred, y_test))
    print(10*'=')

n_features = 16
              precision    recall  f1-score   support

    negative       0.50      0.54      0.52     25845
    positive       0.59      0.55      0.57     30864

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.55     56709

n_features = 64
              precision    recall  f1-score   support

    negative       0.55      0.57      0.56     26876
    positive       0.60      0.58      0.59     29833

    accuracy                           0.58     56709
   macro avg       0.58      0.58      0.58     56709
weighted avg       0.58      0.58      0.58     56709

n_features = 256
              precision    recall  f1-score   support

    negative       0.60      0.61      0.60     27402
    positive       0.63      0.62      0.62     29307

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61  